# Session Analysis

In [1]:
import pandas as pd
import dask.dataframe as ddf
import seaborn as sbn
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import helper_fns
%matplotlib inline

# Initial data exploration

In [2]:
df = ddf.read_csv('session_data_v2.csv')
df.head()

,sess_id,rtt_ms,engaged,conv_val,conv_cnt,conv_rate,start_hour,user,loyalty,loyal,...,os_id,refer_catg,refer_catg_id,refer_subCatg,refer_subCatg_id,pg_count,hit_evnt_cnt,hit_evnt_clicks,hit_evnt_forms,hit_evnt_ajax
0,4347814,100.0,True,0,0,NaN,23,4337070,1,False,...,8,Link,7,NaN,42,1,12,1,1,4
1,4347831,50.0,True,0,0,NaN,23,4337087,1,False,...,12,Search,6,yahoo.com,38,1,11,1,1,3
2,4347833,50.0,True,0,0,NaN,23,4337089,1,False,...,15,Social,5,facebook.com,17,1,8,1,1,4
3,4347837,100.0,True,0,0,NaN,23,4337093,1,False,...,8,Social,5,instagram.com,19,1,8,1,1,2
4,4347838,100.0,True,0,0,NaN,23,4337094,1,False,...,8,Social,5,instagram.com,19,1,11,1,1,3


In [3]:
df.columns

Index(['sess_id', 'rtt_ms', 'engaged', 'conv_val', 'conv_cnt', 'conv_rate',
       'start_hour', 'user', 'loyalty', 'loyal', 'country_iso_code', 'lat',
       'long', 'device_name', 'device_id', 'browser', 'browser_id', 'os_name',
       'os_id', 'refer_catg', 'refer_catg_id', 'refer_subCatg',
       'refer_subCatg_id', 'pg_count', 'hit_evnt_cnt', 'hit_evnt_clicks',
       'hit_evnt_forms', 'hit_evnt_ajax'],
      dtype='object')

In [4]:
df.compute().shape

(2414594, 28)

In [5]:
print('{} unique country codes: {}'.format(len(df.country_iso_code.compute().unique()), df.country_iso_code.compute().unique()))
print('{} unique browser ids: {}'.format(len(df.browser.compute().unique()), df.browser.compute().unique()))
print('{} unique os ids: {}'.format(len(df.os_name.compute().unique()), df.os_name.compute().unique()))
print('{} unique device names: {}'.format(len(df.device_name.compute().unique()), df.device_name.compute().unique()))
print('{} unique referral categories: {}'.format(len(df.refer_catg.compute().unique()), df.refer_catg.compute().unique()))
print('{} unique referral SubCategories: {}'.format(len(df.refer_subCatg.compute().unique()), df.refer_subCatg.compute().unique()))

19 unique country codes: ['US' 'RU' 'DE' 'GB' 'FR' 'IN' 'EG' 'IL' 'IE' 'CA' 'CR' 'UA' 'BR' 'GR'
 'CH' 'PK' 'SE' 'CO' 'FI']
70 unique browser ids: ['Chrome' 'Safari' 'Flipboard App' 'Opera Mobile' 'Google App'
 'Samsung Browser' 'IE' 'Silk' 'Edge' 'Apple Mail' 'UC Browser' 'AliApp'
 'Opera' 'Facebook App' 'Sogou Explorer' 'Yandex Browser'
 'Android WebView' 'Chromium' 'Firefox for iOS' 'Maxthon'
 'Default Browser' 'Vivaldi' 'Whale Browser' 'Splash' 'Iron'
 'Facebook Messenger' 'Firefox' 'Vivo Browser' 'Mobile Safari UIWebView'
 'QQBrowser' 'Dragon' 'liebao' 'Yandex' 'Headless Chrome'
 'Coc Coc Browser' 'BingPreview' 'Google Lighthouse' 'safari' 'chrome'
 'edge' 'samsung browser' 'opera mobile' 'silk' 'ie' 'opera' 'aliapp'
 'chromium' 'vivaldi' 'kuaiso' 'sklikbot' 'feedhub' 'Kuaiso' 'w3clinemode'
 'LinkedInBot' 'SklikBot' 'Desktop Sidebar' 'Nokia Proxy Browser' 'mozdex'
 'apusbrowser' 'webringchecker' 'linkedinbot' 'newsarticle app'
 'magpie-crawler' 'APUSBrowser' 'juzi browser'
 'php br

# Select columns we want to use

In [6]:
# not choosing rtt_ms b.c. it has too many NaNs, not choosing engaged because it only has 1 value
subset_df = df[['conv_rate', 'start_hour', 'loyalty', 'loyal', 'country_iso_code', 'device_name', 'browser', 'os_name', 'refer_catg', 'refer_subCatg', 'pg_count', 'hit_evnt_cnt', 'hit_evnt_clicks', 'hit_evnt_forms', 'hit_evnt_ajax']]
subset_df.head()

,conv_rate,start_hour,loyalty,loyal,country_iso_code,device_name,browser,os_name,refer_catg,refer_subCatg,pg_count,hit_evnt_cnt,hit_evnt_clicks,hit_evnt_forms,hit_evnt_ajax
0,NaN,23,1,False,US,iPad,Chrome,NaN,Link,NaN,1,12,1,1,4
1,NaN,23,1,False,US,Macintosh,Chrome,macOS,Search,yahoo.com,1,11,1,1,3
2,NaN,23,1,False,US,Windows Desktop,Chrome,Windows 8.1,Social,facebook.com,1,8,1,1,4
3,NaN,23,1,False,US,iPad,Safari,NaN,Social,instagram.com,1,8,1,1,2
4,NaN,23,1,False,US,iPhone,Chrome,NaN,Social,instagram.com,1,11,1,1,3


In [7]:
subset_df.compute().shape

(2414594, 15)

# Initial data cleanup

In [8]:
# set converion rates to 0 if originally NaN
subset_df['conv_rate'] = subset_df['conv_rate'].fillna(0)
subset_df = subset_df.dropna()

## noticing some redundancies in OS types so need to lowercase os column
subset_df['os_name'] = subset_df.os_name.apply(lambda x: x.lower())
subset_df['device_name'] = subset_df.device_name.apply(lambda x: x.lower())
subset_df['browser'] = subset_df.browser.apply(lambda x: x.lower())
subset_df['refer_catg'] = subset_df.refer_catg.apply(lambda x: x.lower())
subset_df['refer_subCatg'] = subset_df.refer_subCatg.apply(lambda x: x.lower())

# convert booleans to numerical
subset_df['loyal'] = subset_df.loyal.astype(int)

subset_df.head()

,conv_rate,start_hour,loyalty,loyal,country_iso_code,device_name,browser,os_name,refer_catg,refer_subCatg,pg_count,hit_evnt_cnt,hit_evnt_clicks,hit_evnt_forms,hit_evnt_ajax
1,0.0,23,1,0,US,macintosh,chrome,macos,search,yahoo.com,1,11,1,1,3
2,0.0,23,1,0,US,windows desktop,chrome,windows 8.1,social,facebook.com,1,8,1,1,4
8,0.0,23,1,0,US,linux desktop,chrome,linux,search,google.com,1,6,1,1,3
11,0.0,23,1,0,US,macintosh,chrome,macos,social,pinterest.com,1,6,1,1,2
12,0.0,23,1,0,US,macintosh,chrome,macos,search,baidu.com,1,5,1,1,3


In [9]:
subset_df.compute().describe()

,conv_rate,start_hour,loyalty,loyal,pg_count,hit_evnt_cnt,hit_evnt_clicks,hit_evnt_forms,hit_evnt_ajax
count,1.435733e+06,1.435733e+06,1.435733e+06,1.435733e+06,1.435733e+06,1.435733e+06,1.435733e+06,1.435733e+06,1.435733e+06
mean,4.512901e+03,1.209896e+01,1.121039e+00,5.479222e-02,5.554950e+00,2.306046e+01,1.630660e+01,6.060232e+00,1.203214e+01
std,2.323651e+04,6.864304e+00,3.092361e+00,2.275743e-01,3.918382e+00,1.523124e+01,1.596434e+01,4.491721e+00,7.472736e+00
min,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,0.000000e+00,6.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,7.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
50%,0.000000e+00,1.300000e+01,1.000000e+00,0.000000e+00,5.000000e+00,2.500000e+01,1.300000e+01,5.000000e+00,1.300000e+01
75%,0.000000e+00,1.800000e+01,1.000000e+00,0.000000e+00,9.000000e+00,3.400000e+01,2.700000e+01,1.000000e+01,1.700000e+01
max,3.309960e+05,2.300000e+01,2.910000e+02,1.000000e+00,2.500000e+01,1.090000e+02,1.350000e+02,2.700000e+01,6.000000e+01


# Encode categorical data with OneHot encodings

In [10]:
subset_df = helper_fns.perform_one_hot_encoding(subset_df, 'country_iso_code')
subset_df = helper_fns.perform_one_hot_encoding(subset_df, 'device_name')
subset_df = helper_fns.perform_one_hot_encoding(subset_df, 'browser')
subset_df = helper_fns.perform_one_hot_encoding(subset_df, 'refer_catg')
subset_df = helper_fns.perform_one_hot_encoding(subset_df, 'refer_subCatg')
subset_df = subset_df.drop(columns=['os_name', 'country_iso_code', 'device_name', 'browser', 'refer_catg', 'refer_subCatg'])

subset_df.head()

,conv_rate,start_hour,loyalty,loyal,pg_count,hit_evnt_cnt,hit_evnt_clicks,hit_evnt_forms,hit_evnt_ajax,"(BR,)",...,"(social,)","(ask.com,)","(baidu.com,)","(bing.com,)","(duckduckgo.com,)","(facebook.com,)","(google.com,)","(instagram.com,)","(pinterest.com,)","(yahoo.com,)"
1,0.0,23,1,0,1,11,1,1,3,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,23,1,0,1,8,1,1,4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,0.0,23,1,0,1,6,1,1,3,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
11,0.0,23,1,0,1,6,1,1,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12,0.0,23,1,0,1,5,1,1,3,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Save dataset for future, more efficient, model development and analysis

In [11]:
subset_df.compute().to_csv('session_data_v2_cleaned.csv', index=False)